In [1]:
!wget -O wiki_sample.txt https://raw.githubusercontent.com/dscape/spell/master/test/resources/big.txt
# !wget -O thai_sample.txt https://th.wikipedia.org/wiki/%E0%B8%9B%E0%B8%B1%E0%B8%8D%E0%B8%8D%E0%B8%B2%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%94%E0%B8%B4%E0%B8%A9%E0%B8%90%E0%B9%8C
# (or another small wiki-like article)


--2025-12-04 03:39:14--  https://raw.githubusercontent.com/dscape/spell/master/test/resources/big.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6488666 (6.2M) [text/plain]
Saving to: ‘wiki_sample.txt’

wiki_sample.txt     100%[===================>]   6.19M  --.-KB/s    in 0.09s   

2025-12-04 03:39:14 (65.9 MB/s) - ‘wiki_sample.txt’ saved [6488666/6488666]



In [2]:
# !wget -O attn_paper.pdf https://arxiv.org/pdf/1706.03762.pdf


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# รันครั้งเดียว
!pip install --quiet transformers accelerate sentencepiece huggingface_hub \
    sentence-transformers faiss-cpu PyPDF2 nltk


In [5]:
import os, json, re, math, numpy as np
import faiss, torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")
nltk.download("omw-1.4")

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Device: cpu


[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
import re
from PyPDF2 import PdfReader

# ---------- TXT Reader ----------
def read_text(path):
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

# ---------- PDF Reader ----------
def read_pdf(path):
    reader = PdfReader(path)
    pages = []
    for p in reader.pages:
        try:
            pages.append(p.extract_text() or "")
        except:
            pages.append("")
    return "\n".join(pages)

# ---------- NLTK Sentence Splitter (English) ----------
import nltk
from nltk.tokenize import sent_tokenize

def ensure_nltk():
    # punkt
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')
    # punkt_tab (NLTK >=3.8)
    try:
        nltk.data.find('tokenizers/punkt_tab')
    except LookupError:
        nltk.download('punkt_tab')

def text_to_chunks_by_sentence(text, max_words=300, overlap_words=60):
    ensure_nltk()
    sents = sent_tokenize(text)
    chunks = []
    cur, cur_len = [], 0
    for s in sents:
        w = s.split()
        if cur_len + len(w) > max_words and cur:
            chunks.append(" ".join(cur).strip())
            # overlap
            if overlap_words > 0:
                tail = " ".join(" ".join(cur).split()[-overlap_words:])
                cur = tail.split()
                cur_len = len(cur)
            else:
                cur, cur_len = [], 0
        cur.extend(w)
        cur_len += len(w)
    if cur:
        chunks.append(" ".join(cur).strip())
    return chunks

# ---------- Lightweight splitter (Thai / fallback) ----------
def split_sentences_light(text):
    # no lookbehind → safe for all Python versions
    return [s.strip() for s in re.split(r'[.!?]\s+|\n+', text) if s.strip()]

def text_to_chunks_light(text, max_words=250, overlap_words=50):
    sents = split_sentences_light(text)
    chunks = []
    cur, cur_len = [], 0
    for s in sents:
        w = s.split()
        if cur_len + len(w) > max_words and cur:
            chunks.append(" ".join(cur).strip())
            if overlap_words > 0:
                tail = " ".join(" ".join(cur).split()[-overlap_words:])
                cur = tail.split()
                cur_len = len(cur)
            else:
                cur, cur_len = [], 0
        cur.extend(w)
        cur_len += len(w)
    if cur:
        chunks.append(" ".join(cur).strip())
    return chunks

print("Cell C loaded successfully. PDF + TXT + English + Thai chunkers ready.")


Cell C loaded successfully. PDF + TXT + English + Thai chunkers ready.


In [7]:
# paths
eng_path = "/content/wiki_sample.txt"
thai_path = "/content/20241125-Generative-AI-Guideline_V2-0.pdf"

# โหลดไฟล์ภาษาอังกฤษ
if eng_path.lower().endswith(".pdf"):
    eng_text = read_pdf(eng_path)
else:
    eng_text = read_text(eng_path)

# โหลดภาษาไทย
if thai_path.lower().endswith(".pdf"):
    thai_text = read_pdf(thai_path)
else:
    thai_text = read_text(thai_path)

print("English text length:", len(eng_text))
print(eng_text[:800])

print("\n---\nThai text length:", len(thai_text))
print(thai_text[:800])


English text length: 6488666
The Project Gutenberg EBook of The Adventures of Sherlock Holmes
by Sir Arthur Conan Doyle
(#15 in our series by Sir Arthur Conan Doyle)

Copyright laws are changing all over the world. Be sure to check the
copyright laws for your country before downloading or redistributing
this or any other Project Gutenberg eBook.

This header should be the first thing seen when viewing this Project
Gutenberg file.  Please do not remove it.  Do not change or edit the
header without written permission.

Please read the "legal small print," and other information about the
eBook and Project Gutenberg at the bottom of this file.  Included is
important information about your specific rights and restrictions in
how the file may be used.  You can also find out about how to make a
donation to Project Gutenberg,

---
Thai text length: 59518
0 
 
 Generative AI Governance  
Guideline for Organizations  
 
 
 
 
 
  
1 Generative AI Governance  
Guideline for Organizations  
 
สารบ

In [8]:
# English chunks
eng_chunks = text_to_chunks_by_sentence(eng_text, max_words=200, overlap_words=40)
print("Eng chunks:", len(eng_chunks))
print("Eng chunk[0]:", eng_chunks[0][:300])

# Thai chunks (light splitter)
thai_chunks = text_to_chunks_light(thai_text, max_words=200, overlap_words=40)
print("Thai chunks:", len(thai_chunks))
print("Thai chunk[0]:", thai_chunks[0][:300])


Eng chunks: 7406
Eng chunk[0]: The Project Gutenberg EBook of The Adventures of Sherlock Holmes by Sir Arthur Conan Doyle (#15 in our series by Sir Arthur Conan Doyle) Copyright laws are changing all over the world. Be sure to check the copyright laws for your country before downloading or redistributing this or any other Project
Thai chunks: 31
Thai chunk[0]: 0 Generative AI Governance Guideline for Organizations 1 Generative AI Governance Guideline for Organizations สารบัญ บทสรุปผู้บริหาร Executive Summary 4 01 ท าความเข้าใจ Generative AI What is Generative AI 7 1.1 ค ำนิยำม 9 1.2 ควำมหมำย ของ Generative AI 10 1.3 ควำมสำมำรถของ Generative AI 12 02 ประโย


In [9]:
# load embedder
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
print("Embedder ready")

# function to build index
def build_faiss_index_from_chunks(chunks, embedder, batch_size=32):
    vecs = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i+batch_size]
        em = embedder.encode(batch, convert_to_numpy=True, show_progress_bar=False)
        vecs.append(em)
    vecs = np.vstack(vecs).astype('float32')
    faiss.normalize_L2(vecs)
    d = vecs.shape[1]
    index = faiss.IndexFlatIP(d)
    index.add(vecs)
    return index, vecs

# build for both
eng_index, eng_vecs = build_faiss_index_from_chunks(eng_chunks, embed_model)
thai_index, thai_vecs = build_faiss_index_from_chunks(thai_chunks, embed_model)
print("Eng index size:", eng_index.ntotal, "Thai index size:", thai_index.ntotal)


Embedder ready
Eng index size: 7406 Thai index size: 31


In [10]:
def retrieve(index, embedder, chunks, query, top_k=10):
    if index.ntotal == 0:
        return []
    qvec = embedder.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(qvec)
    k = min(top_k, index.ntotal)
    D, I = index.search(qvec, k)
    results = []
    for score, idx in zip(D[0], I[0]):
        if idx < 0 or idx >= len(chunks):
            continue
        results.append((chunks[idx], float(score), int(idx)))
    return results

def mmr(query_embedding, doc_embeddings, docs, top_k=5, lambda_param=0.6):
    sim_to_query = np.dot(doc_embeddings, query_embedding)
    selected = []
    not_selected = list(range(len(docs)))
    if not not_selected:
        return []
    first = int(np.argmax(sim_to_query))
    selected.append(first)
    not_selected.remove(first)
    while len(selected) < min(top_k, len(docs)):
        mmr_score = []
        for idx in not_selected:
            relevance = sim_to_query[idx]
            diversity = max(np.dot(doc_embeddings[idx], doc_embeddings[s]) for s in selected)
            score = lambda_param * relevance - (1-lambda_param) * diversity
            mmr_score.append((score, idx))
        mmr_score.sort(reverse=True)
        chosen = mmr_score[0][1]
        selected.append(chosen)
        not_selected.remove(chosen)
    return selected


In [11]:
query_eng = "Can you tell me about The Adventures of Sherlock Holmes"
candidates = retrieve(eng_index, embed_model, eng_chunks, query_eng, top_k=10)
print("Candidates (eng):")
for i,(txt,score,idx) in enumerate(candidates):
    print(i, "score", round(score,3), "idx", idx, "preview:", txt[:200].replace("\n"," "))

# MMR rerank (use eng_vecs)
cand_indices = [c[2] for c in candidates]
cand_embs = eng_vecs[cand_indices]
qvec = embed_model.encode([query_eng], convert_to_numpy=True).astype('float32')
# normalize
from numpy.linalg import norm
qvec = qvec[0] / (norm(qvec[0]) + 1e-12)
sel_positions = mmr(qvec, cand_embs, [eng_chunks[i] for i in cand_indices], top_k=4, lambda_param=0.6)
selected_indices = [cand_indices[pos] for pos in sel_positions]
print("Selected (MMR):", selected_indices)
for idx in selected_indices:
    print("-> chunk", idx, eng_chunks[idx][:300].replace("\n"," "))


Candidates (eng):
0 score 0.701 idx 1 preview: to make a donation to Project Gutenberg, and how to get involved. **Welcome To The World of Free Plain Vanilla Electronic Texts** **eBooks Readable By Both Humans and By Computers, Since 1971** *****T
1 score 0.669 idx 471 preview: you go to the official police." "Oh, I have heard of that fellow," answered my visitor, "and I should be very glad if he would take the matter up, though of course I must use the official police as we
2 score 0.636 idx 57 preview: Holmes. "You have but to name it." "This photograph!" The King stared at him in amazement. "Irene's photograph!" he cried. "Certainly, if you wish it." "I thank your Majesty. Then there is no more to 
3 score 0.635 idx 342 preview: for some days." "That was it," said Holmes, nodding approvingly; "I have no doubt of it. But have you never been prosecuted for begging?" "Many times; but what was a fine to me?" "It must stop here, h
4 score 0.615 idx 377 preview: town suppliers. Now, look 

In [12]:
query_th = "องค์ประกอบสำคัญของ AI มีอะไรบ้าง?"
candidates = retrieve(thai_index, embed_model, thai_chunks, query_th, top_k=10)
print("Candidates (thai):")
for i,(txt,score,idx) in enumerate(candidates):
    print(i, "score", round(score,3), "idx", idx, "preview:", txt[:200].replace("\n"," "))

cand_indices = [c[2] for c in candidates]
cand_embs = thai_vecs[cand_indices]
qvec = embed_model.encode([query_th], convert_to_numpy=True).astype('float32')
qvec = qvec[0] / (np.linalg.norm(qvec[0]) + 1e-12)
sel_positions = mmr(qvec, cand_embs, [thai_chunks[i] for i in cand_indices], top_k=4, lambda_param=0.6)
selected_indices = [cand_indices[pos] for pos in sel_positions]
print("Selected (MMR):", selected_indices)
for idx in selected_indices:
    print("-> chunk", idx, thai_chunks[idx][:300].replace("\n"," "))


Candidates (thai):
0 score 0.37 idx 24 preview: ำหนดกลยุทธ์ และบริหำรจัดกำรควำมเสี่ยง เพิ่มเติม • ภำยหลัง ก ำหนดกรณีกำรใช้งำน (Use case) และเป้ำหมำยในกำ ประยุกต์ใช้ Generative AI แล้ว องค์กรควรก ำหนดรูปแบบกำรน ำ Generative AI มำประยุกต์ใช้ ที่เหมำะ
1 score 0.369 idx 4 preview: วิดีโอ ซอร์สโค้ด หรือรูปแบบอื่น เป็นต้น ด้วยกำรสั่ง กำรผ่ำนข้อควำมหรือค ำสั่ง ( Prompt) ที่มนุษย์เป็นผู้ก ำหนด เมื่อ Generative AI ได้รับ Prompt จำกผู้ใช้งำน แล้ว Generative AI จะท ำ กำรสร้ำงเนื้อหำให
2 score 0.369 idx 13 preview: และผลกระทบ ( Impact ) ที่อำจเกิดขึ้นในมิติต่ำง ๆ ดังนั้น องค์กร ควรหมั่นติดตำมข้อมูลที่เกี่ยวข้องกับเทคโนโลยี และกำรพัฒนำของ Generative AI เพื่อน ำมำทบทวนแนวทำงจัดกำร ควำมเสี่ยงที่อำจเกิดขึ้นจำกกำรประ
3 score 0.362 idx 15 preview: ก ำหนดมำตรกำร และเฝ้ำระวัง ควำมมั่นคงปลอดภัยทำงไซเบอร์ (Establish Cyber Security Mechanisms ) จัดท ำมำตรกำร กำรรักษำควำมปลอดภัย ทำงไซเบอร์ เพื่อป้องกันกำรเข้ำถึง ข้อมูลและระบบ โดยไม่ได้รับอนุญำต (Unau
4 score 0.36 idx 11 preview: หรือรำยละเอียดเหตุกำรณ์ทำ ง ปร

In [13]:
# loader (safe)
FALLBACK_SEQ2SEQ = "google/flan-t5-small"

def load_llm(model_id="google/gemma-2b-it", token=None):
    try:
        tok = AutoTokenizer.from_pretrained(model_id, token=token)
        m = AutoModelForCausalLM.from_pretrained(model_id, token=token)
        if getattr(m.config, 'pad_token_id', None) is None:
            m.config.pad_token_id = m.config.eos_token_id
            tok.pad_token = tok.eos_token
        m.to(device)
        return tok, m, 'causal'
    except Exception as e:
        print("Causal load failed:", e)
    # fallback
    tok = AutoTokenizer.from_pretrained(FALLBACK_SEQ2SEQ)
    m = AutoModelForSeq2SeqLM.from_pretrained(FALLBACK_SEQ2SEQ)
    if getattr(m.config, 'pad_token_id', None) is None:
        m.config.pad_token_id = m.config.eos_token_id
        tok.pad_token = tok.eos_token
    m.to(device)
    return tok, m, 'seq2seq'

token = os.environ.get('HF_TOKEN', None)  # or None if not logged in
tokenizer, model, model_type = load_llm(token=token)
print("Model type:", model_type)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model type: causal


In [14]:
def build_rag_prompt_from_indices(question, chunks, indices, role='story teller', max_context_chars=2000):
    context = ''
    for idx in indices:
        context += f"[chunk {idx}]\\n" + chunks[idx].strip() + '\\n\\n---\\n\\n'
        if len(context) > max_context_chars:
            break
    prompt = f"You are an expert {role}. Use the following context to answer the question. If the answer is not in the context, say 'I don't know'.\\n\\nContext:\\n{context}\\nQuestion: {question}\\n\\nAnswer concisely."
    return prompt

# Build prompt for English
prompt_eng = build_rag_prompt_from_indices(query_eng, eng_chunks, selected_indices, role='Story teller')
print("PROMPT (eng) preview:\\n", prompt_eng[:800])

# Generate
def generate_from_llm_prompt(prompt, tokenizer, model, model_type='causal', max_new_tokens=150):
    if model_type == 'causal':
        inputs = tokenizer(prompt, return_tensors='pt', truncation=True).to(device)
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.2, top_p=0.95)
        text = tokenizer.decode(out[0], skip_special_tokens=True)
        return text[len(prompt):].strip() if text.startswith(prompt) else text
    else:
        inputs = tokenizer(prompt, return_tensors='pt', truncation=True).to(device)
        out = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
        text = tokenizer.decode(out[0], skip_special_tokens=True)
        return text

ans_eng = generate_from_llm_prompt(prompt_eng, tokenizer, model, model_type=model_type, max_new_tokens=150)
print("=== Answer (eng) ===\\n", ans_eng)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


PROMPT (eng) preview:\n You are an expert Story teller. Use the following context to answer the question. If the answer is not in the context, say 'I don't know'.\n\nContext:\n[chunk 24]\n"Was the photograph a cabinet?" "It was." "Then, good-night, your Majesty, and I trust that we shall soon have some good news for you. And good-night, Watson," he added, as the wheels of the royal brougham rolled down the street. "If you will be good enough to call to-morrow afternoon at three o'clock I should like to chat this little matter over with you." II. At three o'clock precisely I was at Baker Street, but Holmes had not yet returned. The landlady informed me that he had left the house shortly after eight o'clock in the morning. I sat down beside the fire, however, with the intention of awaiting him, however long he migh
=== Answer (eng) ===\n I cannot provide a summary of The Adventures of Sherlock Holmes from the context, as the context does not provide any information about the book.


In [15]:
prompt_th = build_rag_prompt_from_indices(query_th, thai_chunks, selected_indices, role='ผู้เชี่ยวชาญด้าน AI')
print("PROMPT (thai) preview:\\n", prompt_th[:800])

ans_th = generate_from_llm_prompt(prompt_th, tokenizer, model, model_type=model_type, max_new_tokens=150)
print("=== Answer (thai) ===\\n", ans_th)


PROMPT (thai) preview:\n You are an expert ผู้เชี่ยวชาญด้าน AI. Use the following context to answer the question. If the answer is not in the context, say 'I don't know'.\n\nContext:\n[chunk 24]\nำหนดกลยุทธ์ และบริหำรจัดกำรควำมเสี่ยง เพิ่มเติม • ภำยหลัง ก ำหนดกรณีกำรใช้งำน (Use case) และเป้ำหมำยในกำ ประยุกต์ใช้ Generative AI แล้ว องค์กรควรก ำหนดรูปแบบกำรน ำ Generative AI มำประยุกต์ใช้ ที่เหมำะสม เช่น กำรเลือกโซลูชันแบบ Off- the-Shelf ที่มีในตลำด กำรเลือกใช้เทคนิค RAG กำรเลือกใช้วิธี กำร Fine-Tuning หรือกำรเลือกท ำ Pre-training เป็นต้น • องค์กรควรมีกำรประเมินควำมเสี่ยง พร้อมทั้ง ก ำหนดมำตรกำรบริหำร ควำมเสี่ยง ที่เกี่ยวข้องกับ Generative AI (ที่ถูกกล่ำวไว้ ในบทที่ 3) ให้อยู่ ในระดับที่ยอมรับได้ (Risk Appetite) • ควรก ำหนดระดับกำรท ำงำนร่วมกันระหว่ำงมนุษย์และ Generative AI ให้สอดคล้อง กับระดับควำมเสี่ยงและผลกระทบ เชิ
=== Answer (thai) ===\n The context does not specify what the elements of AI are, so I cannot answer this question from the context.


In [28]:
# ============================
# Module 3.2 — Retrieval Improvement (One-Cell Version)
# ============================
import numpy as np

# ---- Auto Top-K ----
def auto_top_k(num_chunks):
    k = int(np.log2(num_chunks) * 2)
    return max(5, min(k, 50))  # ไม่ต่ำกว่า 5 ไม่เกิน 50

# ---- Similarity Threshold ----
def filter_by_similarity(results, threshold=0.2):
    # results: [(text, score, idx)]
    return [r for r in results if r[1] >= threshold]

# ---- Expand Query ----
def expand_queries(query):
    return [
        query,
        query.lower(),
        query.replace("what", "").strip(),
        "Explain " + query,
        query + " details"
    ]

# ---- Multi-Query Retrieval ----
def multi_query_retrieve(query, index, embed_model, chunks, top_k):
    qs = expand_queries(query)
    all_results = []
    for q in qs:
        res = retrieve(index, embed_model, chunks, q, top_k)
        all_results.extend(res)

    # merge unique idx
    unique = {}
    for txt, score, idx in all_results:
        if idx not in unique or score > unique[idx][1]:
            unique[idx] = (txt, score)

    merged = [(t, s, i) for i, (t, s) in unique.items()]
    merged.sort(key=lambda x: x[1], reverse=True)
    return merged

# ---- MMR Rerank ----
def mmr_rerank(query, results, embed_model, chunks, top_k=5, lambda_param=0.5):
    if len(results) == 0:
        return []

    # embed query
    query_vec = embed_model.encode([query], convert_to_numpy=True).astype("float32")

    # embed candidate chunks
    cand_texts = [r[0] for r in results]
    cand_vecs = embed_model.encode(cand_texts, convert_to_numpy=True).astype("float32")

    selected = []
    not_selected = list(range(len(cand_texts)))

    # pick highest similarity first
    first = int(np.argmax(np.dot(cand_vecs, query_vec.T)))
    selected.append(first)
    not_selected.remove(first)

    while len(selected) < min(top_k, len(cand_texts)):
        mmr_scores = []
        for i in not_selected:
            relevance = np.dot(cand_vecs[i], query_vec.T)
            diversity = max(np.dot(cand_vecs[i], cand_vecs[j]) for j in selected)
            score = lambda_param * relevance - (1 - lambda_param) * diversity
            mmr_scores.append((score, i))

        mmr_scores.sort(reverse=True)
        chosen = mmr_scores[0][1]
        selected.append(chosen)
        not_selected.remove(chosen)

    return [results[i][2] for i in selected]  # return chunk indices

print("Module 3.2 loaded successfully! 🚀")


Module 3.2 loaded successfully! 🚀


In [29]:
query = "What cases did Sherlock Holmes investigate?"

top_k = auto_top_k(len(eng_chunks))

# multi-query
raw_results = multi_query_retrieve(query, index, embed_model, eng_chunks, top_k)

# filter low similarity
filtered = filter_by_similarity(raw_results, threshold=0.2)

# MMR rerank
selected_indices = mmr_rerank(query, filtered, embed_model, eng_chunks, top_k=5)

print("Selected chunk indices:", selected_indices)


Selected chunk indices: [168, 396, 175, 2, 397]
